## Парсинг данных с сайта на Python

### Определения

Парсинг - это процесс сбора данных с последующей их обработкой и анализом.

Программа, которая занимается парсингом, называют - парсер.


### Условие задачи

С сайта ( https://habr.com/ru/search/ ) необходимо построить исходный набор данных (.csv или .xml). Набор данных должен включать __названия, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статей из Интернет-ресурсов__. Подготовленный набор данных должен содержать сведения о всех номинантах конкурса. Разработанный парсер должен извлекать гиперссылки из начальной страницы с последующим обходом всех страниц по полученным ссылкам и извлечением их содержимого. Можно дополнить набор какими-либо другими данными, если они могут быть полезны для дальнейшего исследования.


### Этапы парсинга

1. Поиск данных
2. Получение информации
3. Сохранение данных

### Подключение библиотек

In [1]:
from bs4 import BeautifulSoup as bs

Beautiful Soup - это библиотека Python для извлечения данных из HTML и XML файлов. 

In [2]:
import requests

Библиотека requests является стандартным инструментом для составления HTTP-запросов в Python.

In [3]:
import pandas as pd

### Получение информаций

In [4]:
# GET - запрос
url = 'https://habr.com/ru/all/' # страница со всеми статьями 
page = requests.get(url)

Метод __requests.get(url)__ из библиотеки requests в Python выполняет HTTP-запрос типа GET по указанному URL. Этот запрос используется для получения данных с веб-страницы или API, в нашем случае из страницы habr.

In [5]:
page.status_code

200

Если вызвать __page.status_code__, то получим статус состояния HTTP. например, 200 — успешно, 404 — страница не найдена, 500 — ошибка сервера 


In [6]:
soup = bs(page.text, 'html.parser')

__bs__ — это сокращение от BeautifulSoup, основного класса библиотеки Beautiful Soup.

__bs(page.text, 'html.parser')__ создаёт объект BeautifulSoup, который парсит HTML-код из page.text с использованием указанного парсера.

__'html.parser'__ — это встроенный парсер Python, который не требует установки дополнительных библиотек.а.

In [7]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n\n  <head>\n    <title>Все статьи подряд &#x2F; Хабр</title>\n<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">\n<link href="https://habr.com/ru/articles/" rel="canonical" data-hid="e3fa780">\n<link href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate" data-hid="7d51b8a">\n<link href="https://habr.com/en/articles/" hreflang="en" rel="alternate" data-hid="7d51b8a">\n<meta itemprop="image" content="/img/habr_ru.png">\n<meta property="og:image" content="/img/habr_ru.png">\n<meta property="og:image:width" content="1200">\n<meta property="og:image:height" content="630">\n<meta property="aiturec:image" content="/img/habr_ru.png">\n<meta name="twitter:image" content="/img/habr_ru.png">\n<meta property="vk:image" content="/img/habr_ru.png?format=vk">\n<meta property="fb:app_id" content="444736788986613">\n<meta property="fb:pages" content="472597926099084">\n<meta name="twitter:card" content="summary_large_image">\n<meta name="tw

In [8]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<title>Все статьи подряд / Хабр</title>
<link data-hid="2a79c45" href="/img/habr_ru.png" rel="image_src"/>
<link data-hid="e3fa780" href="https://habr.com/ru/articles/" rel="canonical"/>
<link data-hid="7d51b8a" href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate"/>
<link data-hid="7d51b8a" href="https://habr.com/en/articles/" hreflang="en" rel="alternate"/>
<meta content="/img/habr_ru.png" itemprop="image"/>
<meta content="/img/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="630" property="og:image:height"/>
<meta content="/img/habr_ru.png" property="aiturec:image"/>
<meta content="/img/habr_ru.png" name="twitter:image"/>
<meta content="/img/habr_ru.png?format=vk" property="vk:image"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@habr_com" name=

Создадим словарь, в который будем записывать данные по заданию: название статьи, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статьи из Интернет-ресурса

In [9]:
result_list = {'Заголовок': [], 'Название компании': [], 'Описание': [], 'Рейтинг': [], 'Категория': [], 'Дата': [], 'Текст': []}

### Алгоритм

Суть алгоритма заключается в переборе страниц, и переходе на "вложенные" страницы, то есть у нас есть основная страница https://habr.com/ru/all/, мы перебираем несколько стараниц с page1 до page10. На каждой странице есть статьи, записываем их в список, чтобы перейти по ним используем  _-i.a.get('href')-_  то есть берём значение из href этого заголовка. Далее находим классы элементов которые нам нужны, и записываем их в результат.

In [10]:
pagenum = 1
for i in range(10):
    url = 'https://habr.com/ru/articles/page' + str(pagenum) + '/' # переход на ссылуку с определённым номером сраницы
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    titles = soup.find_all('h2', class_='tm-title tm-title_h2')# получаем заголовки всех статей на этой странице
    print(titles)
    
    for i in titles: 
        # переход на страницу статьи
        url = 'https://habr.com' + str(i.a.get('href')) 
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        
        name_company = soup.find('a', class_='tm-company-snippet__title')# получаем название компаний
        desc_company = soup.find('div', class_='tm-company-snippet__description')# получаем описание компаний
        
        if (name_company is not None): #если на странице присутсвует компания
        
            result_list['Заголовок'].append(i.text) # записываем название статьи
            result_list['Название компании'].append(name_company.text) # записываем название компании
            result_list['Описание'].append(desc_company.text) # записываем описание компании
            
            datepub = soup.find('span', class_='tm-article-datetime-published') # находим дату публикаций
            result_list['Дата'].append(datepub.time['datetime'][0: 10]) # записываем дату публикаций
            
            # текст статьи
            try:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            except:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            result_list['Текст'].append(textpub)
            
            # переход на страницу компании
            url = 'https://habr.com' + str(name_company.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
                                                                                                                  
            #записываем рейтинг                tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating
            rating = soup.find('span', class_='tm-votes-lever__score-counter tm-votes-lever__score-counter_rating tm-votes-lever__score-counter')
            if(rating is None):
                result_list['Рейтинг'].append('0')
            else:
                result_list['Рейтинг'].append((rating.text).strip())
               
             #записываем отрасли компаний
            fieldtext = ""
            fields = soup.find_all('a', 'tm-company-profile__categories-text')
            for field in fields:
                fieldtext = fieldtext + ((field.text).strip()) + ", "
            if (fields is None):
                result_list['Категория'].append(None)
            else:
                result_list['Категория'].append(fieldtext[0:-2])
            
    pagenum += 1

[<h2 class="tm-title tm-title_h2" data-test-id="articleTitle"><!--[--><a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/articles/898584/"><span>Архитектура в Laravel. Как сделать код понятным и масштабируемым</span></a><!--]--></h2>, <h2 class="tm-title tm-title_h2" data-test-id="articleTitle"><!--[--><a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/articles/898174/"><span>Уже пора менять зимние шины на летние или нет?</span></a><!--]--></h2>, <h2 class="tm-title tm-title_h2" data-test-id="articleTitle"><!--[--><a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/articles/898578/"><span>Умный прицел из 1944-го. Один из первых серийных аналоговых вычислителей</span></a><!--]--></h2>, <h2 class="tm-title tm-title_h2" data-test-id="articleTitle"><!--[--><a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-t

In [11]:
result_list["Рейтинг"]

['80.62',
 '506.66',
 '195.65',
 '0',
 '506.66',
 '3049',
 '662.85',
 '42.58',
 '92.63',
 '3049',
 '96.88',
 '243.06',
 '6.59',
 '2277.26',
 '396.84',
 '506.66',
 '36.13',
 '109.91',
 '579.41',
 '93.77',
 '54.67',
 '37.11',
 '96.74',
 '136.13',
 '1809.43',
 '3049',
 '51.8',
 '75.28',
 '2554.24',
 '202.65',
 '79.57',
 '506.66',
 '2277.26',
 '361.76',
 '68.8',
 '542.26',
 '231.28',
 '1809.43',
 '150.09',
 '2554.24',
 '3049',
 '58.65',
 '99.46',
 '655.92',
 '2277.26',
 '231.28',
 '243.06',
 '1809.43',
 '3049',
 '97.51',
 '237.41',
 '164.12',
 '506.66',
 '3049',
 '50.83',
 '65.4',
 '2554.24',
 '2277.26',
 '68.8']

In [12]:
print("Количество пустых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество пустых значений в: 
Заголовок - 0
Название компании - 0
Описание - 0
Рейтинг - 0
Категория - 0
Дата - 0
Текст - 0


### Сохранение данных

In [13]:
file_name = 'habr.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [14]:
df

,Заголовок,Название компании,Описание,Рейтинг,Категория,Дата,Текст
0,Современная многофакторная аутентификация на п...,Avanpost,Безопасность начинается с управления доступом,80.62,"Программное обеспечение, Информационная безопа...",2025-04-07,"Привет, Хабр! Это Андрей Шпаков, руководитель ..."
1,Как livenessProbe может убить ваш Pod,OTUS,Цифровые навыки от ведущих экспертов,506.66,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2025-04-07,"Привет, Хабр!Если вы хоть раз дебажили под, ко..."
2,Security Week 2515: уязвимости в загрузчике GRUB2,«Лаборатория Касперского»,"Ловим вирусы, исследуем угрозы, спасаем мир",195.65,Программное обеспечение,2025-04-07,На прошлой неделе компания Microsoft отчиталас...
3,Возвращаем доступ к Аниме за пару минут,Amnezia VPN,Software for self-hosted VPN,0,,2025-04-07,"Всем привет! Мы уверены, что среди читателей Х..."
4,Оптимизация бизнес-процессов и ее проблемы,OTUS,Цифровые навыки от ведущих экспертов,506.66,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2025-04-07,В современном деловом мире технологии играют к...
5,13 законов разработки ПО,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,3049,"Связь и телекоммуникации, Домены и хостинг, Ве...",2025-04-07,"Некоторые из них хорошо известны, другие — до..."
6,Java Digest #23,Т-Банк,Компания,662.85,"Программное обеспечение, Электронная коммерция...",2025-04-07,Всем привет! 👋👋👋👋👋 Мы Java-разработчики Т-Банк...
7,Kubernetes для всех: поймет даже гуманитарий,Лаборатория Числитель,Компания,42.58,Программное обеспечение,2025-04-07,Вместо эпиграфа: Статья написана для новичков ...
8,"Фантастическая стратегия тестирования и места,...",АО «ГНИВЦ»,Драйвер цифровой трансформации,92.63,"Веб-разработка, Дизайн и юзабилити, Веб-сервисы",2025-04-07,"Привет, дорогой читатель! Меня зовут Мария Вав..."
9,"Забытые технологии хостинга: 10 решений, котор...",RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,3049,"Связь и телекоммуникации, Домены и хостинг, Ве...",2025-04-07,"Когда мы говорим о современном хостинге, на ..."


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Заголовок          59 non-null     object
 1   Название компании  59 non-null     object
 2   Описание           59 non-null     object
 3   Рейтинг            59 non-null     object
 4   Категория          59 non-null     object
 5   Дата               59 non-null     object
 6   Текст              59 non-null     object
dtypes: object(7)
memory usage: 3.4+ KB


In [16]:
df.to_csv('habr.csv', index=False, encoding='utf-8-sig')